In [3]:
import pandas as pd
from collections import Counter

import shutil
import os

import shutil

import random

## Veriler ile ilgili bilgi edindiğimiz kısım

In [2]:
# CSV dosyasını oku
csv_path = "Data_Entry_2017.csv"
df = pd.read_csv(csv_path)


# Sadece tek bir etiketi olanları filtrele
single_label_df = df[df["Finding Labels"].str.contains(r"\|", na=False) == False]

# Hastalık etiketlerini say
label_counts = Counter(single_label_df["Finding Labels"].dropna())  # Boş olmayanları al



# # Birden çok hastalık etiketlerini de say 
# label_counts = Counter()
# for labels in df["Finding Labels"].dropna():  # Boş olmayanları al
#     label_counts.update(labels.split("|"))  # "|" ile ayrılanları ayır ve say

# Etiketleri çoktan aza doğru sırala
sorted_labels = label_counts.most_common()

# Sonuçları yazdır
print(f"Toplam {len(label_counts)} farklı hastalık etiketi var.")
print("Hastalık etiketleri (çoktan aza doğru):")
for label, count in sorted_labels:
    print(f"{label}: {count}")

Toplam 15 farklı hastalık etiketi var.
Hastalık etiketleri (çoktan aza doğru):
No Finding: 60361
Infiltration: 9547
Atelectasis: 4215
Effusion: 3955
Nodule: 2705
Pneumothorax: 2194
Mass: 2139
Consolidation: 1310
Pleural_Thickening: 1126
Cardiomegaly: 1093
Emphysema: 892
Fibrosis: 727
Edema: 628
Pneumonia: 322
Hernia: 110


## Verileri Türlerine göre ayırdığımız kısım

In [ ]:
def organize_images(df, source_folder = "archive/", destination_folder= "dataset/", chose = 1):
    """
    Görüntüleri etiketlere göre organize eden bir fonksiyon.

    Bu fonksiyon, verilen bir DataFrame'deki görüntü dosyalarını etiketlerine göre
    belirtilen hedef klasöre taşır veya kopyalar. Ayrıca, tüm görüntüleri tek bir
    klasöre toplama seçeneği sunar.

    Args:
        df (pd.DataFrame): Görüntü dosyalarının bilgilerini içeren DataFrame.
                           "Image Index" sütunu dosya adlarını, "Finding Labels" sütunu ise etiketleri içermelidir.
        source_folder (str): Kaynak klasörün yolu. Varsayılan olarak "archive/".
        destination_folder (str): Hedef klasörün yolu. Varsayılan olarak "dataset/".
        chose (int): İşlem türünü belirler:
                     - 1: Görüntüleri etiketlerine göre ayrı klasörlere taşır/kopyalar.
                     - 0: Tüm görüntüleri tek bir klasöre taşır/kopyalar.

    Returns:
        None

    Örnek:
        Tek etiketli görüntüleri organize etmek için:
        >>> organize_images(single_label_df, source_folder="archive/", destination_folder="dataset/", chose=1)
    """

    if chose == 0:
        # Genel hedef klasör (eğer chose = 0 ise tüm görüntüler buraya gider)
        general_folder = os.path.join(destination_folder, "single_label_images")

    for folder in os.listdir(source_folder):
        images_path = os.path.join(source_folder, folder, "images")

        if os.path.exists(images_path):  # Eğer images klasörü varsa

            for filename in os.listdir(images_path):  # İçindeki tüm görüntüleri tarayalım
                file_path = os.path.join(images_path, filename)  # Dosyanın tam yolu
                

                # CSV'den uygun etiket bulunuyor
                row = df.loc[df["Image Index"] == filename]  
                if not row.empty:
                    
                    if chose == 1:
                        label = row["Finding Labels"].values[0]  # Hastalık etiketi

                        # Etikete göre hedef klasör oluştur
                        label_folder = os.path.join(destination_folder, label)
                        os.makedirs(label_folder, exist_ok=True)

                        # Dosyayı taşı
                        destination_path = os.path.join(label_folder, filename)
                        shutil.copy(file_path, destination_path)


                    elif chose == 0:
                        os.makedirs(general_folder, exist_ok=True)

                        # Tüm görüntüleri tek klasöre kopyala
                        destination_path = os.path.join(general_folder, filename)
                        shutil.copy(file_path, destination_path)


    print("Tüm görüntüler etiketlere göre başarıyla taşındı!")

### Tek etiketli görüntüleri organize et

In [ ]:
organize_images(single_label_df)

### Tek etiketli görüntüleri tek bir klasöre kopyala

In [ ]:
organize_images(single_label_df, destination_folder= "single_dataset/", chose= 0)

## Model'de kullanılacak verileri ayırma

In [ ]:
def egitim_dosya_hazirla(source_folder= 'dataset/',
                         destination_folder= "data_model/",
                         excluded_folder = '',
                         num_images_per_class = 1000,
                         train_split_ratio = 0.85,
                         test_split_ratio = 0.10
                         ):
    

    os.makedirs(destination_folder, exist_ok=True)
    disease_folders = [d for d in os.listdir(source_folder)]



    for disease_name in disease_folders:
        # Hariç tutulan klasörü atla
        if disease_name == excluded_folder:
            continue

        disease_path = os.path.join(source_folder, disease_name)

        all_images = [f for f in os.listdir(disease_path)]

        num_available_images = len(all_images)
        if num_available_images < num_images_per_class:

            print(f" Uyarı: '{disease_name}' klasöründe yeterli resim yok.")
            print(f" Mevcut resim sayısı: {num_available_images}, gerekli resim sayısı: {num_images_per_class}")
            print("generated_images klasörü oluşturulacak ve klasör buraya kopyalanacak.")
            print("-" * 20, "\n\n\n")


            # hedef klasörünü oluştur
            os.makedirs(f"generated_images/{disease_name}", exist_ok=True)
            
            # Klasörün içeriğini kopyala
            for image_filename in all_images:
                source_file_path = os.path.join(disease_path, image_filename)
                dest_file_path = os.path.join("generated_images", disease_name, image_filename)
                
                shutil.copy(source_file_path, dest_file_path)
            continue

        

        # Rastgele 1000 resim seç
        selected_images = random.sample(all_images, num_images_per_class)  

        # Seçilen resimleri karıştır
        random.shuffle(selected_images)

        # Bölme için resim sayılarını hesapla
        num_train = int(num_images_per_class * train_split_ratio)
        num_test = int(num_images_per_class * test_split_ratio)
        # Küsüratları doğrulama setine ekleyerek tam sayıya tamamla
        num_val = num_images_per_class - num_train - num_test

        # Resimleri bölümlere ayır
        train_images = selected_images[:num_train]
        test_images = selected_images[num_train:num_train + num_test]
        val_images = selected_images[num_train + num_test:]
    
        # Hedef klasörleri oluştur
        split_mapping = {
        'train': train_images,  # Eğitim seti
        'validation': val_images, # Doğrulama seti
        'test': test_images     # Test seti
        }

        for split_name, image_list in split_mapping.items():

            dest_dir = os.path.join(destination_folder, split_name, disease_name)
            os.makedirs(dest_dir, exist_ok=True) # İç içe klasörleri oluşturur

            # Resimleri kopyalama işlemi
            for image_filename in image_list:
                source_file_path = os.path.join(disease_path, image_filename)
                dest_file_path = os.path.join(dest_dir, image_filename)
                
                shutil.copy(source_file_path, dest_file_path) # Sadece dosyayı kopyalar
                

In [8]:
egitim_dosya_hazirla()

 Uyarı: 'Edema' klasöründe yeterli resim yok.
 Mevcut resim sayısı: 628, gerekli resim sayısı: 1000
generated_images klasörü oluşturulacak ve klasör buraya kopyalanacak.
 Uyarı: 'Emphysema' klasöründe yeterli resim yok.
 Mevcut resim sayısı: 892, gerekli resim sayısı: 1000
generated_images klasörü oluşturulacak ve klasör buraya kopyalanacak.
 Uyarı: 'Fibrosis' klasöründe yeterli resim yok.
 Mevcut resim sayısı: 727, gerekli resim sayısı: 1000
generated_images klasörü oluşturulacak ve klasör buraya kopyalanacak.
 Uyarı: 'Hernia' klasöründe yeterli resim yok.
 Mevcut resim sayısı: 110, gerekli resim sayısı: 1000
generated_images klasörü oluşturulacak ve klasör buraya kopyalanacak.
 Uyarı: 'Pneumonia' klasöründe yeterli resim yok.
 Mevcut resim sayısı: 322, gerekli resim sayısı: 1000
generated_images klasörü oluşturulacak ve klasör buraya kopyalanacak.
